# Lemmaus

Lemmaus tehdään TurkuNLP:n koodin mukaisesti. Lataa ensin tämä kooditiedosto sekä R:ssä luotu lemmattava_txt-tiedosto omaan Google Drive -kansioosi.

Yleisohjeet: https://turkunlp.org/Turku-neural-parser-pipeline/


Huom. Lemmaus on paljon laskutehoa vaativa operaatio. Omalla kohdallani Google epäili kyseessä olevan kielletty toiminta ja lakkautti ensimmäisen lemmausta varten tekemäni googletilin. Kannattanee siis ajaa koodi yksi koodipätkä kerrallaan ja seurata koneen ääressä työn etenemistä, jotta välttää tilin aiheettoman lakkauttamisen.

## Asennetaan TurkuNLP

In [ ]:
!git clone https://github.com/TurkuNLP/Turku-neural-parser-pipeline.git
%cd Turku-neural-parser-pipeline

Cloning into 'Turku-neural-parser-pipeline'...
remote: Enumerating objects: 1309, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1309 (delta 206), reused 304 (delta 180), pack-reused 959
Receiving objects: 100% (1309/1309), 385.80 KiB | 12.86 MiB/s, done.
Resolving deltas: 100% (764/764), done.
/content/Turku-neural-parser-pipeline


## Google Colab asennukset

In [ ]:
!python3 -m pip install ufal.udpipe configargparse transformers "OpenNMT-py>=1.2.0" "git+https://github.com/TurkuNLP/diaparser.git@master" "pytorch_lightning<1.5.0" "torchmetrics<=0.7.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/TurkuNLP/diaparser.git (to revision master) to /tmp/pip-req-build-o9mt1m09
  Running command git clone --filter=blob:none --quiet https://github.com/TurkuNLP/diaparser.git /tmp/pip-req-build-o9mt1m09
  Resolved https://github.com/TurkuNLP/diaparser.git to commit 7b8fab02a7a78efcc7a6072b3ed7588a7d881d0b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.1/304.1 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.8/925.8 KB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.2/398.2 KB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 9.9 MB/s eta 0:00:00
  

In [ ]:
!pip3 install "torchmetrics<=0.7.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytorch_lightning

ModuleNotFoundError: ignored

## Suomenkielinen malli (fi_tdt_dia)

In [ ]:
!python3 fetch_models.py fi_tdt_dia

## Ladataan haluttu pipeline

In [ ]:
from tnparser.pipeline import read_pipelines, Pipeline

# What pipelines do we have for the Finnish model?
available_pipelines=read_pipelines("models_fi_tdt_dia/pipelines.yaml")               # {pipeline_name -> its steps}
# This is a dictionary, its keys are the pipelines
print(list(available_pipelines.keys()))
# Instantiate one of the pipelines
p=Pipeline(available_pipelines["parse_plaintext"])

['parse_plaintext', 'tag_plaintext', 'parse_sentlines', 'parse_wslines', 'parse_conllu', 'tokenize', 'parse_noisytext']


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
Lemmatizer device: gpu / 0


## Oman aineiston lemmaus

In [ ]:
!python3 -m pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Seuraavaksi annetaan oikeus oman googletilisi Google Drive -kansioon:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from conllu import parse as conllu_to_tokenlist

Ladataan Google Drive -kansiosta lemmattava_txt-niminen tiedosto lemmattavaksi:

In [ ]:
with open('/content/drive/MyDrive/lemmattava_txt.txt') as f:
  gradun_data = f.readlines()

In [ ]:
gradun_data_vain_tekstit = gradun_data[3::4]
gradun_data_vain_tekstit

['minusta on tässä yhteydessä syytä huomauttaa että kansainvälisen lain mukaanhan se on viime kädessä israelin eli miehittäjävaltion vastuulla vastata väestön hyvinvoinnista miehitetyillä alueilla\n',
 'on kuitenkin hyvin selkeää se että jos varoja ei välttämättä käytetä sen väestön hyvinvoinnin edistämiseen niin silloinhan meidän tulisi muuttaa omaa poliittista suhtautumistamme siihen kuinka se raha sinne kanavoidaan ja kuinka se apu sinne toimitetaan\n',
 'elinvoimainen ihminen ja kannustavat rakenteet resurssiviisas ja hiilineutraali yhteiskunta sekä kestävän hyvinvoinnin ja työn toimintamallit\n',
 'itse en ymmärrä sitä minkä tähden tällaista asiaa ei saa kritisoida että tämä poistetaan tämä pykälä koska eduskunnan ensisijainen tehtävä on ilman muuta ajaa suomen ja suomalaisten hyvinvointia ja etua ja harkita tarkkaan mihin ne rahat käytetään\n',
 'jatkuva muutos kuormittaa ihmisiä valtavasti ja vähentää heidän työhyvinvointiaan\n',
 'elikkä työhyvinvointi pitää kyllä näissä tuotta

In [ ]:
tulokset = []

for i in gradun_data_vain_tekstit:
    parsed=p.parse(i) #parse it
    parsed_as_tokenlist = conllu_to_tokenlist(parsed)
    tulokset.append(parsed_as_tokenlist)

In [ ]:
lemmat = []

for lauseet in tulokset:
  for lause in lauseet:
    lauseenlemmat = []
    for sana in lause:
      lemma = sana["lemma"]
      lauseenlemmat.append(lemma)
    lemmat.append(lauseenlemmat)

In [ ]:
import csv

In [ ]:
with open("/content/drive/MyDrive/lemmattu_aineisto.csv", "w") as f:
  wr = csv.writer(f)
  wr.writerows(lemmat)

Aineiston lemmaus on nyt valmis. Lataa Drive kansiostasi omalle koneellesi edellä luotu csv-tiedosto ("lemmattu_aineisto.csv"). Tallenna tiedosto omalla koneellasi samaan kansioon R-kooditiedoston kanssa. Avaa R-kooditiedosto ja jatka aihemallinnusprosessissa siitä, mihin aiemmin jäätiin.